首先构造树状程序，树状程序相当于下面这个函数：

In [1]:
def func(x,y):
    if x>3:
        return y+5
    else:
        return y-2

构建四个类

In [120]:
from random import random,randint,choice
from copy import deepcopy
from math import log

#一个封装类，成员包括函数名称，函数本身和该函数接受的参数个数
class fwrapper(object):
    def __init__(self,function,childcount,name):
        self.function = function
        self.childcount = childcount
        self.name = name

#对应于函数型节点，以fwrapper类对其进行初始化,当evaluate被调用时，对各子节点进行求值运算，
#然后将函数本身应用于求得结果
class node(object):
    def __init__(self,fw,children):
        self.function = fw.function
        self.name = fw.name
        self.children = children
        
    def evaluate(self,inp):
        #先对对各子节点进行求值运算
        results = [n.evaluate(inp) for n in self.children]
        #将函数本身应用于求得结果
        return self.function(results)
    
    def display(self,indent=0):
        print((' '*indent)+self.name)
        for c in self.children:
            c.display(indent+1)
    
    
#此类对应的节点返回的是由idx指定的参数
class paramnode(object):
    def __init__(self,idx):
        self.idx = idx
        
    def evaluate(self,inp):
        return inp[self.idx]
    
    def display(self,indent=0):
        print('%sp%d'%(' '*indent,self.idx))
    
#返回常量值的节点，仅返回被初始化时所传入的值
class constnode(object):
    def __init__(self,v):
        self.v = v
    
    def evaluate(self,inp):
        return self.v
    
    def display(self,indent=0):
        print('%s%d'%(' '*indent,self.v))
    

#针对节点的操作函数列表 
addw = fwrapper(lambda m: m[0] + m[1],2,'add')

subw = fwrapper(lambda m: m[0] - m[1],2,'subtract')

mulw = fwrapper(lambda m: m[0] * m[1],2,'multiply')

def iffunc(m):
    if m[0]>0: return m[1]
    else: return m[2]
ifw = fwrapper(iffunc,3,'if')

def isgreater(m):
    if m[0]>m[1]:return 1
    else:return 0
gtw = fwrapper(isgreater,2,'isgreater')

flist = [addw,mulw,ifw,gtw,subw]

def exampletree():
    
    return node(ifw,[node(gtw,[paramnode(0),constnode(3)]),
                      node(addw,[paramnode(1),constnode(5)]),
                      node(subw,[paramnode(1),constnode(2)])
                     ]
                 )

def makerandomtree(pc,maxdepth=4,fpr=0.5,ppr=0.6):
    if random()<fpr and maxdepth>0:
        f = choice(flist)
        children = [makerandomtree(pc,maxdepth-1,fpr,ppr)
                   for i in range(f.childcount)]
        return node(f,children)
    elif random()<ppr:
        return paramnode(randint(0,pc-1))
    else:
        return constnode(randint(0,10))
    
def hiddenfunction(x,y):
    return x**2 + 2*y + 3*x +5

def buildhiddenset():
    rows = []
    for i in range(200):
        x = randint(0,40)
        y = randint(0,40)
        rows.append([x,y,hiddenfunction(x,y)])
    return rows

def scorefunction(tree,s):
    dif = 0
    for data in s:
        v = tree.evaluate([data[0],data[1]])
        #将求得的结果与实际结果的差值求和，累加值越小，题解的表现越好
        dif += abs(v-data[2])
    return dif

def mutate(t,pc,probchange=0.1):
    #如果随机概率小于某个值，对节点进行变异
    if random()<probchange:
        return makerandomtree(pc)
    #否则，再次对子节点进行测试
    else:
        result = deepcopy(t)
        if isinstance(t,node):
            result.children = [mutate(c,pc,probchange) for c in t.children]
        return result
 #交叉   
def crossover(t1,t2,probswap=0.7,top=1):
    if random()<probswap and not top:
        return deepcopy(t2)
    else:
        result = deepcopy(t1)
        if isinstance(t1,node) and isinstance(t2,node):
            result.children = [crossover(c,choice(t2.children),probswap,0)
                              for c in t1.children]
        return result

def evolve(pc,popsize,rankfunction,maxgen=500,
          mutationrate=0.1,breedingrate=0.4,pexp=0.7,pnew=0.05):
    
    #返回一个随机数，通常是一个较小的数
    #pexp取值越小，我们得到的随机数越小，该值越大，筛选过程越严格
    def selectindex():
        return int(log(random())/log(pexp))
    
    #创建一个随机的初始种群
    population = [makerandomtree(pc) for i in range(popsize)]
    for i in range(maxgen):
        
        #用一个函数将这一组程序从优到劣的顺序排列
        scores = rankfunction(population)
        print(scores[0][0])
        #差距为零，终止
        if scores[0][0]==0:break
            
        #总能得到两个最优的程序
        newpop = [scores[0][1],scores[1][1]]
        
        #构造下一代
        #popsize初始种群的大小
        while len(newpop)<popsize:
            if random()>pnew:
                newpop.append(mutate(
                    crossover(scores[selectindex()][1],
                             scores[selectindex()][1],
                            #breedingrate发生交叉的概率
                             probswap=breedingrate),
                            #mutationrate发生变异的概率
                            pc,probchange=mutationrate))
                
            else:
                #加入一个随机节点，以增加种群的多样性
                newpop.append(makerandomtree(pc))
                
        population=newpop
    scores[0][1].display()
    return scores[0][1]

def getrankfunction(dataset):
    def rankfunction(population):
        scores = [(scorefunction(t,dataset),t) for t in population]
        scores.sort()
        return scores
    return rankfunction


In [121]:
rf = getrankfunction(buildhiddenset())
evolve(2,500,rf,mutationrate=0.2,breedingrate=0.1,pexp=0.7,pnew=0.1)

TypeError: '<' not supported between instances of 'node' and 'paramnode'

In [40]:
exampletree = exampletree()

In [41]:
type(exampletree)

__main__.node

In [42]:
exampletree.evaluate([2,3])

1

In [43]:
exampletree.evaluate([5,3])

8

在node、paramnode、constnode中分别设计一个display函数展现程序树

In [37]:
exampletree.display()

if
 isgreater
  p0
  3
 add
  p1
  5
 subtract
  p1
  2


In [ ]:
#在node中用来显示出整棵树的字符串形式
def display(self,indent=0):
    print((' '*indent)+self.name)
    for c in self.children:
        c.display(indent+1)

In [ ]:
#打印出该节点返回参数的对应索引即可
def display(self,indent=0):
    print('%sp%d'%(' '*indent,self.idx))

In [ ]:
def display(self,indent=0):
    print('%s%d'%(' '*indent,self.v))

## 构造初始种群

初始种群由一组随机程序构成的

In [56]:
def makerandomtree(pc,maxdepth=4,fpr=0.5,ppr=0.6):
    if random()<fpr and maxdepth>0:
        f = choice(flist)
        children = [makerandomtree(pc,maxdepth-1,fpr,ppr)
                   for i in range(f.childcount)]
        return node(f,children)
    elif random()<ppr:
        return paramnode(randint(0,pc-1))
    else:
        return constnode(randint(0,10))

In [57]:
random1 = makerandomtree(2)
random1.evaluate([7,1])

3

In [58]:
random1.evaluate([2,4])

3

In [59]:
random2 = makerandomtree(2)
random2.evaluate([5,3])

30

In [60]:
random2.evaluate([5,20])

30

In [61]:
random1.display()

3


In [62]:
random2.display()

multiply
 multiply
  p0
  isgreater
   p1
   subtract
    5
    p1
 6


## 测试题解  
一个简单的数学测试

首先构造一个简单的数学函数

In [63]:
def hiddenfunction(x,y):
    return x**2 + 2*y + 3*x +5

然后利用上述函数构造一个数据集

In [64]:
def buildhiddenset():
    rows = []
    for i in range(200):
        x = randint(0,40)
        y = randint(0,40)
        rows.append([x,y,hiddenfunction(x,y)])
    return rows

In [65]:
hiddenset = buildhiddenset()

构造一个简单的测试方法衡量题解的优劣

In [66]:
def scorefunction(tree,s):
    dif = 0
    for data in s:
        v = tree.evaluate([data[0],data[1]])
        #将求得的结果与实际结果的差值求和，累加值越小，题解的表现越好
        dif += abs(v-data[2])
    return dif

In [68]:
scorefunction(random2,hiddenset)

109388

In [69]:
scorefunction(random1,hiddenset)

131804

由于这小部分程序完全随机产生，因此其中存在正确解的概率是非常小的，要等到正确解，下面对程序进行变异和交叉，然后优胜劣汰

## 变异

In [ ]:
def mutate(t,pc,probchange=0.1):
    #如果随机概率小于某个值，对节点进行变异
    if random()<probchange:
        return makerandomtree(pc)
    #否则，再次对子节点进行测试
    else:
        result = deepcopy(t)
        if isinstance(t,node):
            result.children = [mutate(c,pc,probchange) for c in t.children]
        return result

In [71]:
random2.display()

multiply
 multiply
  p0
  isgreater
   p1
   subtract
    5
    p1
 6


尝试执行mutate函数，看看函数如何对数进行修改

In [102]:
muttree = mutate(random2,2)
muttree.display()

multiply
 1
 if
  if
   10
   add
    p0
    p0
   isgreater
    0
    p1
  isgreater
   p1
   p0
  10


接下来看变好还是变差。变异是随机的，并非一定随着变好的方向进行，只希望一部分变异对最终结果有帮助

In [103]:
scorefunction(random2,hiddenset)

109388

In [105]:
scorefunction(muttree,hiddenset)

132284

## 交叉

In [106]:
def crossover(t1,t2,probswap=0.7,top=1):
    if random()<probswap and not top:
        return deepcopy(t2)
    else:
        result = deepcopy(t1)
        if isinstance(t1,node) and isinstance(t2,node):
            result.children = [crossover(c,choice(t2.children),probswap,0)
                              for c in t1.children]
        return result

In [107]:
random1 = makerandomtree(2)
random1.display()

if
 isgreater
  multiply
   7
   if
    p1
    9
    9
  p1
 multiply
  p0
  4
 p0


In [108]:
random2 = makerandomtree(2)
random2.display()

4


In [109]:
cross = crossover(random1,random2)
cross.display()

if
 isgreater
  multiply
   7
   if
    p1
    9
    9
  p1
 multiply
  p0
  4
 p0


## 构筑环境

接下来开始构筑供程序进化的竞争环境，首先生成一组随机程序并择优复制和修改，然后一直重复到条件满足为止

In [116]:
def evolve(pc,popsize,rankfunction,maxgen=500,
          mutationrate=0.1,breedingrate=0.4,pexp=0.7,pnew=0.05):
    
    #返回一个随机数，通常是一个较小的数
    #pexp取值越小，我们得到的随机数越小，该值越大，筛选过程越严格
    def selectindex():
        return int(log(random())/log(pexp))
    
    #创建一个随机的初始种群
    population = [makerandomtree(pc) for i in range(popsize)]
    for i in range(maxgen):
        
        #用一个函数将这一组程序从优到劣的顺序排列
        scores = rankfunction(population)
        print(scores[0][0])
        #差距为零，终止
        if scores[0][0]==0:break
            
        #总能得到两个最优的程序
        newpop = [scores[0][1],scores[1][1]]
        
        #构造下一代
        #popsize初始种群的大小
        while len(newpop)<popsize:
            if random()>pnew:
                newpop.append(mutate(
                    crossover(scores[selectindex()][1],
                             scores[selectindex()][1],
                            #breedingrate发生交叉的概率
                             probswap=breedingrate),
                            #mutationrate发生变异的概率
                            pc,probchange=mutationrate))
                
            else:
                #加入一个随机节点，以增加种群的多样性
                newpop.append(makerandomtree(pc))
                
        population=newpop
    scores[0][1].display()
    return scores[0][1]

In [117]:
def getrankfunction(dataset):
    def rankfunction(population):
        scores = [(scorefunction(t,dataset),t) for t in population]
        scores.sort()
        return scores
    return rankfunction

In [118]:
rf = getrankfunction(buildhiddenset())
evolve(2,500,rf,mutationrate=0.2,breedingrate=0.1,pexp=0.7,pnew=0.1)

TypeError: '<' not supported between instances of 'paramnode' and 'paramnode'